In [114]:
import mysql.connector
import pandas as pd
import numpy as np
import re

In [40]:
mydb = mysql.connector.connect(
    host="localhost",
    user="student",
    password="student")
print(mydb)

In [80]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS proyecto_final")
mycursor.execute("USE proyecto_final")

In [81]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Universidades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                url varchar(255), \
                tipo_centro varchar(255))")

In [82]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Facultades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                id_univ int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [217]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Grados (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_fac int, \
                nombre varchar(255), \
                presencial bool, \
                nota_corte double, \
                idioma bool, \
                nota_corte_ant double, \
                nota_admision double, \
                rendimiento double, \
                creditos double, \
                precio_creditos double, \
                empleabilidad double, \
                FOREIGN KEY (id_fac) REFERENCES Facultades(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [84]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('Facultades',)
('Grados',)
('Universidades',)


In [58]:
df = pd.read_csv("file:/home/student/Downloads/Datos_QEDU.csv")

In [153]:
df2 = df.replace(['Pública / Propio', 'Pública / Adscrito**'], 'Pública').replace(['Privada / Propio', 'Privada / Adscrito**'], 'Privada')
dfUniv = df2.groupby(['NOMBRE_UNIV', 'URL_INFO', 'TIPO_UNIV_CENTRO']).size().reset_index().iloc[:, :-1]
dfUniv

,NOMBRE_UNIV,URL_INFO,TIPO_UNIV_CENTRO
0,Atlántico Medio,https://www.universidadatlanticomedio.es/plane...,Privada
1,CUNEF Universidad,https://www.cunef.edu/estudios/cunef-grado/,Privada
2,ESIC Universidad,https://www.esic.edu/university/grados,Privada
3,Hesperides,https://hesperides.edu.es/estudios/,Privada
4,IE Universidad,http://www.ie.edu/es/universidad/estudios/ofer...,Privada
...,...,...,...
83,Universidad de la Rioja,https://www.unirioja.es/estudios/grados/,Pública
84,Universidad de las Illes Balears,http://estudis.uib.cat/es/grau/,Pública
85,Universidad del País Vasco/Euskal Herriko Unib...,https://www.ehu.eus/es/web/estudiosdegrado-gra...,Pública
86,Universitat Abat Oliba CEU,https://www.uaoceu.es/estudios?tipos_estudio=3...,Privada


In [151]:
sqlUniv = "INSERT INTO Universidades (nombre, url, tipo_centro) VALUES (%s, %s, %s)"
for index, row in dfUniv.iterrows():
    val = (row['NOMBRE_UNIV'], row['URL_INFO'], row['TIPO_UNIV_CENTRO'])
    mycursor.execute(sqlUniv, val)
mydb.commit()

In [155]:
dfFac = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO']).size().reset_index().iloc[:, :-1]
dfFac

,NOMBRE_UNIV,NOMBRE_CENTRO
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas (Las...
1,CUNEF Universidad,Escuela Politécnica Superior (Madrid)
2,CUNEF Universidad,"Facultad de Empresa, Economía y Derecho (Madrid)"
3,ESIC Universidad,Facultad de Ciencias Jurídicas y Sociales (Mad...
4,Hesperides,Escuela de Grados (Las Palmas)
...,...,...
1072,Universitat de València (Estudi General),Facultad de Psicología y Logopedia (Valencia)
1073,Universitat de València (Estudi General),Facultad de Química (Valencia)
1074,Universitat de València (Estudi General),Facultat d'Infermeria i Podologia (Valencia)
1075,Universitat de València (Estudi General),Facultat de Fisioteràpia (Valencia)


In [215]:
sqlFac = "INSERT INTO Facultades (nombre, id_univ) VALUES (%s, %s)"
sqlQueryUniv = "SELECT id FROM Universidades WHERE nombre = %s"
uniAnterior = ""
idUni = -1
for index, row in dfFac.iterrows():
    uniActual = row['NOMBRE_UNIV']
    if uniAnterior != uniActual:
        mycursor.execute(sqlQueryUniv, (uniActual, ))
        myresult = mycursor.fetchall()
        idUni = myresult[0][0]
        uniAnterior = uniActual
    val = (row['NOMBRE_CENTRO'], idUni)
    mycursor.execute(sqlFac, val)
mydb.commit()

In [188]:
dfGrad = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA', 'PRESENCIALIDAD', 'NOTA_CORTE', 'IDIOMAEXTRANJERO',
                      'NOTA_CORTE_ANT','NOTA_ADMISION', 'RENDIMIENTO',
                      'CREDITOS', 'PRECIO', 'TASA_AFIL']).size().reset_index().iloc[:, :-1]
dfGrad

,NOMBRE_UNIV,NOMBRE_CENTRO,NOM_TITULA,PRESENCIALIDAD,NOTA_CORTE,IDIOMAEXTRANJERO,NOTA_CORTE_ANT,NOTA_ADMISION,RENDIMIENTO,CREDITOS,PRECIO,TASA_AFIL
0,Universidad Autónoma de Barcelona,Escuela Universitaria de Enfermería del Hospit...,Grado en Enfermería,Sí,10.03,No,10.76,11.18,95.0,240.0,18.46,84.4
1,Universidad Autónoma de Barcelona,Escuela Universitaria de Enfermería y Terapia ...,Grado en Enfermería,Sí,8.04,No,8.62,9.63,95.0,240.0,18.46,94.2
2,Universidad Autónoma de Barcelona,Escuela de Ingeniería (Barcelona),Grado en Gestión Aeronáutica,Sí,5.15,No,7.77,9.42,85.0,240.0,18.46,80.5
3,Universidad Autónoma de Barcelona,Escuela de Ingeniería (Barcelona),Grado en Ingeniería Informática,Sí,9.70,No,9.32,10.33,76.0,240.0,18.46,84.2
4,Universidad Autónoma de Barcelona,Escuela de Ingeniería (Barcelona),Grado en Ingeniería Química,Sí,8.97,No,8.52,10.12,77.0,240.0,18.46,79.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1276,Universitat de València (Estudi General),Facultad de Psicología y Logopedia (Valencia),Grado en Logopedia,Sí,8.92,No,9.29,9.33,93.0,240.0,18.00,86.1
1277,Universitat de València (Estudi General),Facultad de Psicología y Logopedia (Valencia),Grado en Psicología,Sí,10.43,Sí*,10.51,10.89,90.0,240.0,14.03,60.5
1278,Universitat de València (Estudi General),Facultat d'Infermeria i Podologia (Valencia),Grado en Podología,Sí,11.03,No,11.08,11.08,87.0,240.0,18.00,80.7
1279,Universitat de València (Estudi General),Facultat de Magisteri (Valencia),Grado en Maestro/a en Educación Infantil,Sí,9.59,No,9.75,10.37,96.0,240.0,12.79,77.1


In [219]:
sqlGrad = "INSERT INTO Grados (id_fac, nombre, presencial, nota_corte, idioma, \
                                    nota_corte_ant, nota_admision, rendimiento, creditos, precio_creditos, \
                                    empleabilidad) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sqlQueryFac = "SELECT id FROM Facultades WHERE id_univ = (SELECT id FROM Universidades WHERE nombre = %s) AND nombre = %s"
uniAnterior = ""
facAnterior = ""
idFac = -1
for index, row in dfGrad.iterrows():
    uniActual = row['NOMBRE_UNIV']
    facActual = row['NOMBRE_CENTRO']
    if uniAnterior != uniActual or facAnterior != facActual:
        mycursor.execute(sqlQueryFac, (uniActual, facActual))
        myresult = mycursor.fetchall()
        idFac = myresult[0][0]
        uniAnterior = uniActual
        facAnterior = facActual
    if row['PRESENCIALIDAD']=="No": 
        presencialidad=0 
    else: 
        presencialidad=1
    if row['IDIOMAEXTRANJERO']=="No":
        idioma=0 
    else: 
        idioma=1
    val = (idFac, row['NOM_TITULA'], presencialidad, row['NOTA_CORTE'], 
           idioma, row['NOTA_CORTE_ANT'], row['NOTA_ADMISION'], row['RENDIMIENTO'],
           row['CREDITOS'], row['PRECIO'], row['TASA_AFIL'])
    mycursor.execute(sqlGrad, val)
mydb.commit()

('information_schema',)
('hive',)
('hue',)
('labs',)
('mysql',)
('performance_schema',)
('proyecto_final',)
('prueba1',)
('unis',)


,URL_INFO,NOM_TITULA,NOMBRE_UNIV,NOMBRE_CENTRO,TIPO_UNIV_CENTRO,PRESENCIALIDAD,IDIOMAEXTRANJERO,PLAZAS,NOTA_CORTE,NOTA_CORTE_ANT,NOTA_ADMISION,RENDIMIENTO,CREDITOS,PRECIO,TASA_AFIL
0,https://www.unex.es/organizacion/servicios-uni...,Grado en Educación Social,Universidad de Extremadura,Facultad de Formación del Profesorado (Cáceres),Pública / Propio,Sí,No,65.0,5.93,8.13,10.40,95.0,240.0,11.82,56.2
1,https://www.unex.es/organizacion/servicios-uni...,Grado en Ingeniería Mecánica (Rama Industrial),Universidad de Extremadura,Escuela de Ingenierías Industriales (Badajoz),Pública / Propio,Sí,No,75.0,6.55,6.00,9.99,57.0,240.0,16.13,92.5
2,https://www.ubu.es/grados,Grado en Derecho,Universidad de Burgos,Facultad de Derecho (Burgos),Pública / Propio,Sí,No,80.0,5.00,5.00,7.51,70.0,240.0,12.59,41.7
3,https://www.um.es/web/estudios/grados,Grado en Traducción e Interpretación,Universidad de Murcia,Facultad de Letras (Murcia),Pública / Propio,Sí,No,100.0,7.66,7.39,12.06,90.0,240.0,14.38,NaN
4,https://web.ua.es/es/grados-oficiales.html,Grado en Derecho,Universidad de Alicante,Facultad de Derecho (Alicante),Pública / Propio,Sí,Sí*,240.0,9.11,9.55,10.06,80.0,240.0,11.84,52.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3631,http://www.unir.net/grados-online.aspx,Grado en Historia del Arte,Universidad Internacional de la Rioja,Facultad de Ciencias Sociales y Humanidades (L...,Privada / Propio,No,No,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN
3632,http://estudios.uoc.edu/es/grados,Grado en Administración y Dirección de Empresas,Universidad Oberta de Catalunya,Universitat Oberta de Catalunya (Barcelona),Privada / Propio,Sí,No,1800.0,NaN,NaN,NaN,NaN,240.0,NaN,NaN
3633,http://www.ub.edu/dyn/cms/continguts_es/estudi...,Grado en Diseño y Desarrollo de Videojuegos y ...,Universidad de Barcelona,ENTI - Escuela de Nuevas Tecnologías Interacti...,Pública / Adscrito**,-,No,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN
3634,http://www.usj.es/estudios/grados,Grado en Criminología,Universidad San Jorge,Facultad de Comunicación y Ciencias Sociales (...,Privada / Propio,-,No,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN


233